# Submit phonons calculation

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# General imports.
import ipywidgets as ipw
from IPython.display import clear_output

# AiiDA imports.
%load_ext aiida
%aiida
from aiida import orm, plugins

# AiiDAlab imports.
import aiidalab_widgets_base as awb

# Custom imports.
from surfaces_tools.widgets import build_slab, computational_resources, inputs, empa_viewer, cdxml
from surfaces_tools.utils import wfn

Cp2kPhononsWorkChain = plugins.WorkflowFactory('nanotech_empa.cp2k.phonons')

In [ ]:
# Structure selector.
empa_viewer = empa_viewer.EmpaStructureViewer()
build_slab = build_slab.BuildSlab(title='Build slab')
ipw.dlink((empa_viewer, 'details'), (build_slab, 'details'))
ipw.dlink((empa_viewer, 'structure'), (build_slab, 'molecule'))

structure_selector = awb.StructureManagerWidget(
    viewer=empa_viewer,
    importers=[
        awb.StructureUploadWidget(title="Import from computer"),
        awb.StructureBrowserWidget(title="AiiDA database"),
        awb.OptimadeQueryWidget(embedded=True),
        awb.SmilesWidget(title="From SMILES"),
        cdxml.CdxmlUpload2GnrWidget(title="CDXML"),
    ],
    editors = [
        awb.BasicStructureEditor(title="Edit structure"),
        build_slab
    ],
    storable=False, node_class='StructureData')
display(structure_selector)

# Code.
code_input_widget = awb.ComputationalResourcesWidget(
            description="CP2K code:", default_calc_job_plugin="cp2k"
        )
resources = computational_resources.ProcessResourcesWidget()

protocol = ipw.Dropdown( 
            value="standard",
            options= [
                ("Phonons", "phonons"),
                ("Standard", "standard"),
                ("Low accuracy", "low_accuracy"),
                ("Debug", "debug"),
                ],
            description="Protocol:",
            style={"description_width": "initial"},
        )

input_details = inputs.InputDetails()
input_details.phonons = True
output = ipw.Output()

In [ ]:
ipw.dlink((empa_viewer, 'details'), (input_details, 'details'))
ipw.dlink((code_input_widget, 'value'),(input_details, 'selected_code'))

def prepare_inputs():
    with output:
        clear_output()
    if not structure_selector.structure_node:
        can_submit, msg = False, 'Select a structure first.'
    elif not code_input_widget.value:
        can_submit, msg = False, 'Select CP2K code.'
    else:  
        can_submit, msg, parameters = input_details.return_final_dictionary()
    
    if not can_submit:
        with output:
            print(msg)
            return

    builder = Cp2kPhononsWorkChain.get_builder()
    builder.metadata.label = "CP2K_Phonons"
    builder.protocol = orm.Str(protocol.value)
    builder.metadata.description = parameters['description']    
    builder.code = orm.load_code(code_input_widget.value)
    builder.options = {
            "max_wallclock_seconds": resources.walltime_seconds,
            "withmpi": True,
            "resources": {
                'num_machines': resources.nodes,
                'num_mpiprocs_per_machine': resources.tasks_per_node,
                'num_cores_per_mpiproc': resources.threads_per_task,
            },
        }
    
    builder.structure = structure_selector.structure_node
    builder.dft_params = orm.Dict(parameters['dft_params'])
    builder.sys_params = orm.Dict(parameters['sys_params'])
    builder.phonons_params = orm.Dict(parameters['phonons_params'])

    # Check if a restart wfn is available.
    wave_function = None
    if structure_selector.structure_node.is_stored:
        wave_function = wfn.structure_available_wfn(
        node=structure_selector.structure_node,
        relative_replica_id=None,
        current_hostname=builder.code.computer.hostname,
        return_path=False,
        dft_params=parameters['dft_params'],
        )
    if wave_function is not None:
        print(f"Restarting from wfn in folder: {wave_function.pk}")
        builder.parent_calc_folder = wave_function    
   
    return builder

In [ ]:
btn_submit = awb.SubmitButtonWidget(
    Cp2kPhononsWorkChain,
    inputs_generator=prepare_inputs,
    disable_after_submit=False,
    append_output=True,
    )

In [ ]:
# Resources estimation.
resources_estimation_button = computational_resources.ResourcesEstimatorWidget()
resources_estimation_button.link_to_resources_widget(resources)
ipw.dlink((structure_selector, 'structure'), (input_details, 'ase_atoms'))
ipw.dlink((empa_viewer, 'details'), (resources_estimation_button, 'details'))
ipw.dlink((input_details, 'uks'), (resources_estimation_button, 'uks'))
ipw.dlink((input_details, 'phonons'),(resources, 'phonons'))
ipw.dlink((input_details, 'n_replica_trait'),(resources, 'n_replica_trait'))
ipw.dlink((resources, 'nproc_replica_trait'),(input_details, 'nproc_replica_trait'))
_ = ipw.dlink((code_input_widget, 'value'), (resources_estimation_button, 'selected_code'))

# Inputs

In [ ]:
display(input_details, protocol)

# Code and resources

In [ ]:
display(code_input_widget, resources, resources_estimation_button)

# Submit

In [ ]:
display(btn_submit, output)